# Cartpole Reinforcement Learning

### Importing the necessary Modules

In [204]:
import gym
import numpy as np

from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Flatten,Input,Concatenate,Activation
from tensorflow.keras.optimizers import Adam

from rl.agents import DDPGAgent
from rl.policy import BoltzmannQPolicy
from rl.random import OrnsteinUhlenbeckProcess
from rl.memory import SequentialMemory,EpisodeParameterMemory

### Creating the environement 

In [176]:
env = gym.make("Ant-v4")

c:\Users\TRETEC\anaconda3\envs\ai\lib\site-packages\gym\core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\Users\TRETEC\anaconda3\envs\ai\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


### Analyzing the cartpole environement

In [177]:
states = env.observation_space.shape[0]
actions = env.action_space.shape[0]

In [178]:
print(f"number of states : {states}")
print(f"number of actions : {actions}")


number of states : 27
number of actions : 8


In [179]:
env.observation_space.sample()

array([ 0.77777014,  1.61724424,  0.81562214, -1.02381538,  0.32098166,
       -1.49601233, -0.30379704, -0.36751761, -0.55325822, -2.01112154,
       -0.37974534,  1.79045066,  0.96841525,  1.35483594,  0.49838011,
       -0.22222846,  1.36946265,  0.92481802, -0.66038453, -1.29365243,
        0.96494259,  0.39274802, -0.60784743, -0.81422861,  1.27714252,
        1.09033436, -0.95454529])

In [180]:
env.action_space.sample()

array([-0.8269655 , -0.03337483,  0.7987928 , -0.56918126,  0.07511768,
        0.2451269 , -0.29642606, -0.8184176 ], dtype=float32)

### Testing the Environement on random samples

In [181]:
# episodes = 10
# for episode in range(1,episodes+1):
#     state = env.reset()
#     done = False
#     score=0

#     while not done:
#         action = env.action_space.sample()
#         _,reward,done,_ = env.step(action)
#         score +=reward
#         env.render()

#     print(f"Episode {episode}, score {score}")

# env.close()




### Generating the training the neural network on the environement

In [223]:
def build_model(states,actions):
    model = Sequential([
        Flatten(input_shape=(1,states)),
        Dense(128,activation="relu"),
        Dense(128,activation="relu"),
        Dense(128,activation="relu"),
        Dense(actions,activation="linear"),
        # Reshape((8,)),
    ])
    return model
# model = Sequential([
#     observation_input = Input(shape=(1,) + env.observation_space.shape, name='observation_input')

# # Define your custom neural network model
# x = Flatten()(observation_input)
#     Flatten(input_shape=(1,states)),
#     Dense(64, activation='relu'),
#     Dense(64, activation='relu'),
#     Dense(actions, activation='tanh')
# ])

In [224]:
action_input = Input(shape=(actions,),name="action_input")
observation_input = Input(shape=(1,states),name="observation_input")
flattened_input = Flatten()(observation_input)
x=Concatenate()([action_input,flattened_input])
x=Dense(128)(x)
x=Activation('relu')(x)
x=Dense(128)(x)
x=Activation('relu')(x)
x=Dense(128)(x)
x=Activation('relu')(x)
x=Dense(1)(x)
x=Activation('linear')(x)
critic = Model(inputs=[action_input,observation_input],outputs=x)


In [225]:
actor = build_model(states,actions)

# model.summary()

In [226]:
# Define the memory
memory = SequentialMemory(limit=100000, window_length=1)

random_process = OrnsteinUhlenbeckProcess(size=actions,theta=.15,mu=0.,sigma=.3)

In [227]:
# agent = CEMAgent(model=build_model(), memory=memory, nb_actions=env.action_space.shape[0])
# agent = DQNAgent(model=model, memory=memory, policy=policy,
#                nb_actions=actions, nb_steps_warmup=10,
#                target_model_update=1e-2)
agent = DDPGAgent(nb_actions=actions,actor=actor,critic=critic,critic_action_input=action_input
                  ,memory=memory,nb_steps_warmup_critic=100,nb_steps_warmup_actor=100,random_process=random_process
                  ,gamma=.99,target_model_update=1e-3)

In [228]:
agent.compile(Adam(lr=1e-3), metrics=['mae'])
agent.fit(env,nb_steps=100000,visualize=False,verbose=1)

Training for 100000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 212s 21ms/step - reward: -2.4781
20 episodes - episode_reward: -1181.938 [-8203.627, -135.823] - loss: 9.551 - mae: 1.487 - mean_q: -6.630 - reward_forward: -0.014 - reward_ctrl: -3.464 - reward_survive: 1.000 - x_position: -0.301 - y_position: -0.385 - distance_from_origin: 1.129 - x_velocity: -0.014 - y_velocity: -0.011 - forward_reward: -0.014

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 217s 22ms/step - reward: -1.0538
10 episodes - episode_reward: -1049.401 [-2890.300, -270.792] - loss: 2.516 - mae: 0.594 - mean_q: 1.631 - reward_forward: 0.002 - reward_ctrl: -2.056 - reward_survive: 1.000 - x_position: 0.150 - y_position: 0.169 - distance_from_origin: 0.509 - x_velocity: 0.002 - y_velocity: 0.006 - forward_reward: 0.002

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 220s 22ms/step - reward: -0.7774
11

In [229]:
results =agent.test(env,nb_episodes=10,visualize=True)
print(np.mean(results.history["episode_reward"]))
env.close()


Testing for 10 episodes ...


c:\Users\TRETEC\anaconda3\envs\ai\lib\site-packages\gym\core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


Episode 1: reward: 802.422, steps: 1000
Episode 2: reward: 603.981, steps: 1000
Episode 3: reward: 823.452, steps: 1000
Episode 4: reward: 815.575, steps: 1000
Episode 5: reward: 85.326, steps: 128
Episode 6: reward: 88.748, steps: 171
Episode 7: reward: 824.280, steps: 1000
Episode 8: reward: 537.195, steps: 1000
Episode 9: reward: 727.240, steps: 1000
Episode 10: reward: 659.091, steps: 1000
596.7309379812964
